In [1]:
import pandas as pd
import numpy as np

In [8]:
credit = pd.read_csv("credit.csv")
credit

,index,date,B_value,B_Profit rate,B _credit_of_future_tendency,B_Risk_credit,G_value,G_Profit rate,G_Credit_of_future_tendency,G_Risk_credit,DealDay,Gold_buy_credit,Bit_buy_credit
0,0,2016/9/11,621.65,NaN,0.000000,-1.000000,1324.60,0.000000,NaN,0.000000,0,NaN,0.583818
1,1,2016/9/12,609.67,-0.019271,0.000000,-3.000000,1324.60,0.000000,NaN,0.000000,1,NaN,0.586565
2,2,2016/9/13,610.92,0.002050,0.000000,-2.995511,1323.65,-0.000717,NaN,-3.121320,1,NaN,0.589410
3,3,2016/9/14,608.82,-0.003437,0.000000,-3.474993,1321.75,-0.001435,NaN,-3.449490,1,NaN,0.592347
4,4,2016/9/15,610.38,0.002562,0.000000,-3.380798,1310.80,-0.008284,-1.000000,-3.631432,1,NaN,0.595690
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,1821,2021/9/6,51769.06,0.036472,0.008106,-1.032375,1821.60,-0.001152,-0.101199,0.243126,1,0.408421,0.588687
1822,1822,2021/9/7,52677.40,0.017546,NaN,-1.033989,1802.15,-0.010677,-0.200791,-0.083578,1,0.417782,0.585363
1823,1823,2021/9/8,46809.17,-0.111399,NaN,-1.109223,1786.00,-0.008962,-0.395499,-0.032688,1,0.425504,0.595029
1824,1824,2021/9/9,46078.38,-0.015612,NaN,-1.113038,1788.25,0.001260,-0.596142,-0.002509,1,0.425545,0.676900


In [6]:
credit
bit_signal = credit.Bit_buy_credit

In [15]:
lower_bit = 0.56
higher_bit = 0.71
initial_money = 1000
initial_bit = 0
alpha_bit = 0.98
for idx, b_val in enumerate(credit.B_value):
    if (bit_signal[idx] < lower_bit):
        initial_money += (initial_bit)*(bit_signal[idx])*b_val*alpha_bit
        initial_bit -= (initial_bit)*(bit_signal[idx])
        # print([idx, initial_money,initial_bit, initial_money+initial_bit*b_val])
    elif (bit_signal[idx] > higher_bit):
        initial_bit += (initial_money*(bit_signal[idx])*alpha_bit)/b_val
        initial_money -= initial_money*(bit_signal[idx])
        # print([idx,initial_money,initial_bit,initial_money+initial_bit*b_val])
    

In [ ]:
# 黄金买0.58，卖0.3,中间不动
# 比特币买0.71，卖56
per_gold = 0.01
per_bt = 0.02
init_gold = 0
init_money = 1000
init_bit = 0
for i, (val, Deal, b_val) in enumerate(zip(credit.G_value,gold.DealDay, credit.B_value)):
    bt = bit_signal[i]
    if Deal:  # 判断交易日
        gold = credit.Gold_buy_credit[i]
        if gold > 0.58:
            if bt > 0.71:# 两者都可买入时
                if gold-0.58>(bt-0.71)*2:  # 买入黄金
                    init_gold += init_money*gold*(1-per_gold)/val
                    init_money -= init_money*gold
#                     print('两者选黄金',df['黄金持有份额'][i],df['现金持有份额'][i],df['现金持有份额'][i-1],gold,1-per_gold,df['USD (PM)'][i-1])
                else:  # 买入比特币
                    init_bit+=init_money*bt*(1-per_bt)/b_val
                    init_money-=init_money*bt
#                     print('两者选比特币',df['比特币持有份额'][i],df['现金持有份额'][i])
            else: # 只能买黄金时
                df['黄金持有份额'][i]=df['黄金持有份额'][i-1]+df['现金持有份额'][i-1]*gold*(1-per_gold)/df['USD (PM)'][i-1]
                df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*gold
#                 print('只能买黄金',df['黄金持有份额'][i])
        if gold<0.3:  # 买入黄金
            df['黄金持有份额'][i]=df['黄金持有份额'][i-1]-df['黄金持有份额'][i-1]*(1+gold-0.58)
            df['现金持有份额'][i]=df['现金持有份额'][i-1]+df['黄金持有份额'][i-1]*(1+gold-0.58)*(1-per_gold)*df['USD (PM)'][i-1]
#             print('卖黄金',df['黄金持有份额'][i],df['现金持有份额'][i])
        continue    
    if bt > 0.71:  # 买入比特币
        df['比特币持有份额'][i]=df['比特币持有份额'][i-1]+df['现金持有份额'][i-1]*bt*2*(1-per_bt)/df['Value'][i-1]
        df['现金持有份额'][i]=df['现金持有份额'][i-1]-df['现金持有份额'][i-1]*bt
#         print('买比特币',df['比特币持有份额'][i],df['现金持有份额'][i])
    if bt < 0.56:  # 卖出比特币
        df['比特币持有份额'][i]=df['比特币持有份额'][i-1]-df['比特币持有份额'][i-1]*(1+gold-0.56)
        df['现金持有份额'][i]=df['现金持有份额'][i-1]+df['比特币持有份额'][i-1]*(1+gold-0.56)*(1-per_gold)*df['Value'][i-1]